# OJA Deduplication Challenge

## Imports

### Packages

In [58]:
import os
import warnings

import pandas as pd
from tqdm import tqdm

# from Levenshtein import distance
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [59]:
warnings.filterwarnings('ignore')

### Data

In [60]:
# Import from s3
os.system(f"mc cp s3/apalazzolo/Deduplication/wi_dataset.csv wi_dataset.csv")

`s3/apalazzolo/Deduplication/wi_dataset.csv` -> `wi_dataset.csv`
Total: 0 B, Transferred: 203.23 MiB, Speed: 155.83 MiB/s


0

In [61]:
data = pd.read_csv('wi_dataset.csv',
                   lineterminator='\n')

In [62]:
# For now let's work with a smaller extract

# data = original_data.head(10000)

In [63]:
n_ads = len(data)
n_ads

112056

In [64]:
data.head(3)

,id,title,description,location,country_id,company_name,retrieval_date
0,1,Traineeship Pensioen & Leven in Utrecht,We sturen je door naar je toekomstige opdracht...,Utrecht,NL,NaN,2021-01-19
1,2,DEPOSITARY OFFICER (M/F),DEPOSITARY OFFICER (M/F) DO Recruitment Adviso...,NaN,FR,DO Recruitment Advisors,2021-09-30
2,3,Cautam colegi manipulanti marfa,Descriere Angajam manipulanti marfa din Pitest...,Pitesti,RO,NaN,2021-06-18


## Data preprocessing

In [65]:
# Basic cleaning

data[
    ['title', 'description', 'location', 'company_name']
] = data[
    ['title', 'description', 'location', 'company_name']
].apply(lambda x: x.str.strip().str.lower())

data.fillna("", inplace=True)

## Naive deduplication

In [66]:
duplicates = []

### Add the full duplicates

In [67]:
# Way too long to run
# + sorting the table breaks the code

# for i in tqdm(range(n_ads)):
#     for j in range(i+1,n_ads):
#         if (data.iloc[i, 1] == data.iloc[j, 1]) and (data.iloc[i, 2] == data.iloc[j, 2]):
#             duplicates.append({'id1': i+1, 'id2': j+1, 'type': 'FULL'})

In [68]:
data.sort_values(by=['title', 'description', 'id'], inplace=True)

i = 0
j = 1

while i < n_ads and j < n_ads:
    if (i%10000 == 0) and (j == i+1):
        print(i)
    if data.iloc[i, 1] < data.iloc[j, 1]:
        i += 1
        j = i + 1
    elif data.iloc[i, 1] > data.iloc[j, 1]:
        j += 1
    elif data.iloc[i, 2] < data.iloc[j, 2]:
        i += 1
        j = i + 1
    elif data.iloc[i, 2] > data.iloc[j, 2]:
        j += 1
    else:
        duplicates.append({'id1': data.iloc[i, 0], 'id2': data.iloc[j, 0], 'type': 'FULL'})
        j += 1

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000


In [69]:
len(duplicates)

357497

In [70]:
duplicates[0]

{'id1': 236, 'id2': 1598, 'type': 'FULL'}

### Add the semantic duplicates

In [71]:
data.sort_values(by=['id'], inplace=True)

In [72]:
data['text'] = data['title'] + ' ' + data['description'] + ' ' + data['location'] + ' ' + data['company_name']

# Use TF-IDF to vectorize the texts
vectorizer = TfidfVectorizer()
tfidf = vectorizer.fit_transform(data['text'])
tfidf

<112056x439107 sparse matrix of type '<class 'numpy.float64'>'
	with 15934299 stored elements in Compressed Sparse Row format>

In [73]:
# Use cosine similarity to compare the text
# But actually way too big output

# similarity_matrix = cosine_similarity(tfidf)

In [74]:
# Identify similar ads paris
# Needs to be done by steps

# for i in tqdm(range(similarity_matrix.shape[0])):
#     for j in range(i+1, similarity_matrix.shape[1]):
#         if similarity_matrix[i][j] > 0.8 and (
#             (data.iloc[i, 1] != data.iloc[j, 1]) or (data.iloc[i, 2] != data.iloc[j, 2])
#         ):
#             if data.iloc[i, 6] != data.iloc[j, 6]:
#                 duplicates.append({'id1': data.iloc[i, 0], 'id2': data.iloc[j, 0], 'type': 'TEMPORAL'})
#             else:
#                 if abs(
#                     len(data.iloc[i, 2]) - len(data.iloc[j, 2])
#                 ) / max(
#                     len(data.iloc[i, 2]), len(data.iloc[j, 2])
#                 ) < 0.08:
#                     duplicates.append({'id1': data.iloc[i, 0], 'id2': data.iloc[j, 0], 'type': 'SEMANTIC'})
#                 else:
#                     duplicates.append({'id1': data.iloc[i, 0], 'id2': data.iloc[j, 0], 'type': 'PARTIAL'})

In [75]:
def cosine_similarity_by_chunk(start, end):
    if end > n_ads:
        end = n_ads
    return cosine_similarity(X=tfidf[start:end], Y=tfidf)

In [76]:
chunk_size = 1000

for chunk_start in tqdm(range(0, n_ads, chunk_size)):
    similarity_matrix_chunk = cosine_similarity_by_chunk(chunk_start, chunk_start+chunk_size)
    compteur_init = len(duplicates)
    for i in range(chunk_size):
        for j in range(chunk_start+i+1, n_ads):
            if similarity_matrix_chunk[i][j] > 0.9 and (
                (data.iloc[chunk_start+i, 1] != data.iloc[j, 1]) or (data.iloc[chunk_start+i, 2] != data.iloc[j, 2])
            ):
                if data.iloc[chunk_start+i, 6] != data.iloc[j, 6]:
                    duplicates.append({'id1': data.iloc[chunk_start+i, 0], 'id2': data.iloc[j, 0], 'type': 'TEMPORAL'})
                else:
                    if abs(
                        len(data.iloc[chunk_start+i, 2]) - len(data.iloc[j, 2])
                    ) / max(
                        len(data.iloc[chunk_start+i, 2]), len(data.iloc[j, 2])
                    ) < 0.1:
                        duplicates.append({'id1': data.iloc[chunk_start+i, 0], 'id2': data.iloc[j, 0], 'type': 'SEMANTIC'})
                    else:
                        duplicates.append({'id1': data.iloc[chunk_start+i, 0], 'id2': data.iloc[j, 0], 'type': 'PARTIAL'})
    compteur_end = len(duplicates)
    print(compteur_end-compteur_init)

  1%|          | 1/113 [00:36<1:07:19, 36.07s/it]

48152


  2%|▏         | 2/113 [01:11<1:05:43, 35.53s/it]

44927


  3%|▎         | 3/113 [01:46<1:04:58, 35.44s/it]

...

859


 99%|█████████▉| 112/113 [37:30<00:04,  4.63s/it]

218


100%|██████████| 113/113 [37:31<00:00, 19.93s/it]

1


## Print duplicates

In [77]:
duplicates = pd.DataFrame(duplicates)
duplicates.sort_values(by=['id1', 'id2'], inplace=True)
duplicates

,id1,id2,type
357497,1,29699,TEMPORAL
357498,1,72192,SEMANTIC
53379,3,86,FULL
53380,3,11168,FULL
357499,3,27623,SEMANTIC
...,...,...,...
2871056,111777,111956,TEMPORAL
2871057,111777,111975,TEMPORAL
290641,111869,112054,FULL
2871058,111956,111975,TEMPORAL


In [78]:
duplicates.groupby('type').count()

,id1,id2
type,,
FULL,357497,357497
PARTIAL,1238,1238
SEMANTIC,212506,212506
TEMPORAL,2299819,2299819


In [81]:
duplicates.to_csv('duplicates.csv', index=False, header=False)

In [82]:
os.system(f"mc cp duplicates.csv s3/apalazzolo/Deduplication/duplicates.csv")

`/home/onyxia/work/deduplication/duplicates.csv` -> `s3/apalazzolo/Deduplication/duplicates.csv`
Total: 0 B, Transferred: 56.15 MiB, Speed: 144.77 MiB/s


0